# Work with Facebook API in Python
---

---

<img src="images/logo.png" width="55%">

The Facebook API is a platform for building applications that are available to the members of the social network of [Facebook](https://www.facebook.com/). The API allows applications to use the social connections and profile information to make applications more involving, and to publish activities to the news feed and profile pages of Facebook, subject to individual users privacy settings. With the API, users can add social context to their applications by utilizing profile, friend, page, group, photo, and event data. The API uses RESTful protocol and responses are in JSON format.

## Register a new application

If you do not already have an account with Facebook, go to http://facebook.com and register. You need also register as a Facebook Developer here http://developers.facebook.com by clicking of the "Register" button at the right top corner of the page

<img src="images/app_1.jpg">

After that from http://developer.facebook.com, click on "My Apps" at the top of the page to go to the application dashboard or use the link https://developers.facebook.com/apps. The dashboard shows a list of applications that the developer has created. If you haven’t created any applications yet, then the list will be empty. Click the "Create a New App" button near the top. Note, you can name your application almost anything, although Facebook does apply a few rules. For example, the name can’t contain the words "Face", "fb" or "Book" (in various register).

<img src="images/app_2.jpg">

A dialog prompts you to name your application. You are asked to supply an App Namespace. You use the App Namespace to define custom Open Graph actions and as part of the application’s Facebook URL (http://apps.facebook.com/{App_Namespace}). Make sure the App Namespace is longer than 7 characters, contains no capital letters, and is unique among all applications on Facebook.

<img src="images/app_3.jpg" width="70%">

After you click Continue, Facebook performs a Captcha check to verify that you’re not setting up applications through an automated process. Once you’ve satisfied the verification process, your application is created. The next page you see is your application’s application settings page. 

<img src="images/app_4.jpg">

From the application settings page, you can configure various details about your application. The choices you make here depend on what kind of application you plan to build and what you want your application to do.

The main thing to note from the application settings page is the App ID and App Secret near the top. These values are your application’s credentials to Facebook. You need these credentials to do almost anything with Facebook, including going through the OAuth authorization flow and working with Facebook’s Graph API.

### ACCESS_TOKEN generation

An access token is an opaque string that identifies a user, app, or page and can be used by the app to make graph API calls. Access tokens are obtained via a number of methods, each of which are covered later in this document. There are different types of access tokens to support different use cases:

* **User Access Token:**

The user token is the most commonly used type of token. This kind of access token is needed any time the app calls an API to read, modify or write a specific person's Facebook data on their behalf. User access tokens are generally obtained via a login dialog and require a person to permit your app to obtain one. 

* **App Access Token:**

This kind of access token is needed to modify and read the app settings. It can also be used to publish Open Graph actions. It is generated using a pre-agreed secret between the app and Facebook and is then used during calls that change app-wide settings. You obtain an app access token via a server-to-server call.

* **Page Access Token:**

These access tokens are similar to user access tokens, except that they provide permission to APIs that read, write or modify the data belonging to a Facebook Page. To obtain a page access token you need to start by obtaining a user access token and asking for the manage_pages permission. Once you have the user access token you then get the page access token via the Graph API.

To get access tokens select menu "Tools & Support"

<img src="images/app_5.jpg">

After that you need select "Access Token Tool" point in the left panel "Tools"

<img src="images/app_6.jpg">

In the opened window you may see the App Token of your app. But for us it more interesting the User Token. To create it you should click on "need to grand permission" link and confirm creation of this token. 

<img src="images/app_7.jpg">

The Graph API is the primary way to get data in and out of Facebook's social graph. It's a low-level HTTP-based API that you can use to query data, post new stories, upload photos and a variety of other tasks that an app might need to do. 

The Graph API is named after the idea of a 'social graph' - a representation of the information on Facebook composed of:

* **nodes** - basically "things" such as a User, a Photo, a Page, a Comment;
* **edges** - the connections between those "things", such as a Page's Photos, or a Photo's Comments;
* **fields** - info about those "things", such as a person's birthday, or the name of a Page.

The Graph API is HTTP based, so it works with any language that has an HTTP library, such as `urllib`, `requests` in Python.

> Note, to work with Facebook Graph API you should login!

In [ ]:
import requests

In [ ]:
# Besides, you may get App Access Token using Python by the following way, for example 
def get_fb_app_access_token(app_id, app_secret): 
    """ 
    URL general form:
    https://graph.facebook.com/oauth/access_token?
     client_id={app-id}&
     client_secret={app-secret}&
     grant_type=client_credentials
    """
    
    payload = {'grant_type': 'client_credentials', 'client_id': app_id, 'client_secret': app_secret}
    r = requests.post('https://graph.facebook.com/oauth/access_token?', params = payload)
    print r.status_code
    print r.url
    print r.text    
    result = r.text.split("=")[1]
    return result

app_id = "<YOUR_APP_ID>"
app_secret = "<YOUR_APP_SECRET>"

# Compare this code with those you see here https://developers.facebook.com/tools/access_token/.
print get_fb_app_access_token(app_id, app_secret)

In [ ]:
access_token = "<YOUR_USER_ACCESS_TOKEN>"

# 1. Interraction with Facebook API using `requests` Python library 

In [ ]:
import json
from IPython.core.display import HTML

# At first let's look at your Facebook id, name, link to page and avatar
url_1 = 'https://graph.facebook.com/me?fields=id,name,picture,link&access_token={}'.format(access_token)
    
r_1 = requests.get(url_1)
print r_1.text

# Deserialize the response and write back out as pretty-printed JSON
print json.dumps(json.loads(r_1.content), indent=1)

my_id = json.loads(r_1.content)["id"]
print my_id

HTML('<p><img src="{}" /></p>'.format(json.loads(r_1.content)["picture"]["data"]["url"]))

In [ ]:
# Get all your friends
url_2 = 'https://graph.facebook.com/me?fields=friends&access_token={}'\
        .format(access_token)
    
r_2 = requests.get(url_2)
print json.dumps(json.loads(r_2.content), indent=1)

my_friends = json.loads(r_2.content)["friends"]["data"]

In [ ]:
# Get 10 likes for 10 friends
url_3 = 'https://graph.facebook.com/me?fields=name,friends.limit(10).fields(likes.limit(10))&access_token={}'\
        .format(access_token)
    
r_3 = requests.get(url_3)
print json.dumps(json.loads(r_3.content), indent=1)

In [ ]:
# Check if two people are friends:
# In case the people are not friends, an empty response

def check_friends(user_id_1, user_id_2):
    url = 'https://graph.facebook.com/{}/friends/{}&access_token={}'.format(user_id_1, user_id_2, access_token)
    r = requests.get(url)
    return json.dumps(json.loads(r.content), indent=1)

print check_friends(my_friends[0], my_id), "\n"
print check_friends(my_friends[0], my_friends[1])

In [ ]:
# Get your likes
url_4 = 'https://graph.facebook.com/me?fields=likes&access_token={}'\
        .format(access_token)
    
r_4 = requests.get(url_4)
print json.dumps(json.loads(r_4.content), indent=1)

It was only a small list of examples with basic Facebook API usage. The full list of the Facebook Graph API root nodes and available fields can be found [here](https://developers.facebook.com/docs/graph-api/reference). There are too many variants and rules of queries building and its learning will unlikely fit in even one separated lesson. You may find a little more examples of URLs forming [here](https://developers.facebook.com/docs/graph-api/using-graph-api). A better way is using of specific libraries for interaction with APIs.

# 2. Interaction with Facebook API using specific Python library 

`facebook-sdk` is one of the famous and common used special client library that is designed to support the Facebook Graph API and the official Facebook JavaScript SDK, which is the canonical way to implement Facebook authentication.

The easiest way of installing `facebook-sdk` is using pip:

    pip install facebook-sdk

In [ ]:
import facebook 

# Create a connection to the Graph API with your access token
graph = facebook.GraphAPI(access_token)

# Return data for specific "user_id" of "me" 
me = graph.get_object("me")
me

In [ ]:
# Fetch the connections for given object
friends = graph.get_connections("me", "friends")
friends

In [ ]:
# Search pages with key word "Python"
graph.request("search", {'q' : 'Python', 'type' : 'page'})

In [ ]:
# Use the 'Monty Python' id to query for likes
monty_python_id = '272002449634285'
print "Monty Python likes:", 
graph.get_connections(monty_python_id, "likes")

In [ ]:
# Use the 'Monty Python' id to query for feeds
print "Monty Python feeds:", 
graph.get_connections(monty_python_id, "feed")

In [ ]:
# Write some phrase on your wal
graph.put_object("me", "feed", message="Ha! I've just written this sentence on my wall using Python.")
# Check whether this phrase appeared on your wall.

In [ ]:
# Look at Bill Gates's profile and his posts.
user = 'BillGates'
profile = graph.get_object(user)
print profile

posts = graph.get_connections(profile['id'], 'posts')

for post in posts['data']:
    print post['created_time'], '\t' + post["message"][:100] + '...' if len(post["message"]) > 100 else post["message"]

In [ ]:
from IPython.core.display import HTML

# Look at Bill Gates's profile avatar.
user = 'BillGates'
profile = graph.get_object(user)

avatar = graph.get_connections(profile['id'], 'picture', width=500)

HTML('<p><img src="{}" width="300px"></p>'.format(avatar["url"]))

In [ ]:
# Let's query for all of the likes in your page
likes = {friend['name'] : g.get_connections(friend['id'], "likes")['data'] 
          for friend in friends["data"]}

print likes

In [ ]:
# Let's analyze all likes from friendships for frequency
from prettytable import PrettyTable
from collections import Counter

friends_likes = Counter([like['name'] for friend in likes  for like in likes[friend] if like.get('name')])

pt = PrettyTable(field_names=['Name', 'Freq'])
pt.align['Name'], pt.align['Freq'] = 'l', 'r'
[pt.add_row(fl) for fl in friends_likes.most_common(10)]

print 'Top 10 likes amongst friends'
print pt

In [ ]:
# And now analyze all like categories by frequency
friends_likes_categories = Counter([like['category'] for friend in likesfor like in likes[friend]])

pt = PrettyTable(field_names=['Category', 'Freq'])
pt.align['Category'], pt.align['Freq'] = 'l', 'r'
[pt.add_row(flc) for flc in friends_likes_categories.most_common(10)]

print "Top 10 like categories for friends"
print pt

In [ ]:
# Which of your likes are in common with which friends?
my_likes = [like['name'] for like in g.get_connections("me", "likes")['data']]

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[pt.add_row((ml,)) for ml in my_likes]
print "My likes"
print pt

# Use the set intersection to find common likes
common_likes = list(set(my_likes) & set(friends_likes))

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[pt.add_row((cl,)) for cl in common_likes]
print
print "My common likes with friends"
print pt

# Which of your friends like things that you like?
similar_friends = [(friend, friend_like['name']) 
                       for friend, friend_likes in likes.items()
                           for friend_like in friend_likes
                               if friend_like.get('name') in common_likes]


# Filter out any possible duplicates that could occur
ranked_friends = Counter([ friend for (friend, like) in list(set(similar_friends)) ])

pt = PrettyTable(field_names=["Friend", "Common Likes"])
pt.align["Friend"], pt.align["Common Likes"] = 'l', 'r'
[ pt.add_row(rf) 
  for rf in sorted(ranked_friends.items(), 
                   key=itemgetter(1), 
                   reverse=True) ]
print "My similar friends (ranked)"
print pt

In [ ]:
# A quick histogram that shows above results
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt


plt.hist(ranked_friends.values(),
          bins=arange(1,max(ranked_friends.values()),1))
plt.xlabel('Bins (number of friends with shared likes)')
plt.ylabel('Number of shared likes in each bin')
plt.figure() 

---
## Exercises
---

## Exercise #1: 

> Can you display your avatar in this notebook?

## Exercise #2: 

> `facebook-sdk` GraphAPI method `put_comment(object_id, message)` allows to add a comment text `message` to some post with `object_id`, method `put_like(object_id)` allows add like to an object (post, page, photo, etc.) with `object_id`. Add a comment to some post of some your friend and check the respective changes.

## Exercise #3: 

> Using `facebook-sdk` Python library calculate the number of likes for each your friend and sorting by frequency. 